In [ ]:
from nanorsm_parallel import *

In [ ]:
# select the parameter file to be loaded

params = load_json_file()

In [ ]:
# load elemental image data, align it with pystackreg and generate a tranform matrix

data_path = params['fluorescence data path']
output_path = params['output path']
if not os.path.exists(output_path):
    os.makedirs(output_path)
    print(f"Created directory: {output_path}")

sid_list = params['scan ids']
if params['alignment matrix'] is None:
    elem = params['element for alignment']
    file_list = [
        f"{data_path}output_tiff_scan2D_{sid}//detsum_{elem}_norm.tiff"
        for sid in sid_list
    ]
    im_stack = load_ims(file_list)
    num_frame,im_row,im_col = np.shape(im_stack)
    im_stack_aligned, trans_matrix = align_im_stack(im_stack) # use pystackreg
    im_stack_test = interp_sub_pix(im_stack,trans_matrix) # verify the alignment is done correctly
    slider_view(im_stack_test)
    params['alignment matrix'] = trans_matrix
    print('if you are happy with the alignment, run the cell again')
else:
    elem_list = params['element list']
    trans_matrix = np.asarray(params['alignment matrix'])
    stack = []
    for i, elem in enumerate(elem_list):
        file_list = [
            f"{data_path}output_tiff_scan2D_{sid}//detsum_{elem}_norm.tiff"
            for sid in sid_list
        ]
        im_stack = load_ims(file_list)
        im_stack_test = interp_sub_pix(im_stack, trans_matrix)
        imp = np.sum(im_stack_test, axis=0)
        stack.append(imp[np.newaxis, ...])
        tifffile.imwrite(f"{output_path}{elem}.tiff",im_stack_test.astype(np.float32),imagej=True)
        print(f"save {elem} to {output_path}{elem}.tiff")
    stack = np.concatenate(stack, axis=0)     

In [ ]:
roi = params['detector roi']
[scan_row, scan_col] = params['scan dimensions']
det = params['detector name']
mon = params['monitor']
diff_data = load_h5_data_db_parallel(sid_list,det=det,mon=mon,roi=roi)
sz = diff_data.shape
diff_data = np.reshape(diff_data,(sz[0],scan_row,scan_col,sz[2],sz[3]))
diff_data = interp_sub_pix(diff_data,trans_matrix)

In [ ]:
### transform to cartesian crystal coordinates (z along hkl and x is the rocking direction)

energy = params['energy']
delta = params['delta']
gamma = params['gamma']
num_angle = params['number of angles']
th_step = params['angle step']
pix = params['pixel size']
det_dist = params['detector distance']
offset = np.asarray(params['roi offset'])
data_store = params['data store'] # this will reduce the data stored. If use 'full' it can be over 100G

method = {'fit_type': 'com',# fitting method: center of mass, 'com' or 'peak'
         'shape': 'gaussian', # peak shape: 'gaussian', 'lorentzian', or 'voigt'
         'n_peaks': [1,1,1], # number of peaks in each direction
          'mask': 'tot', # reference image used for creating a mask
          'mask threshold': 0.1 # pixels with values below maximun*thresh of the reference image are set to zero
         } # number of peaks in each direction, qx, qy, and qz


# generate an object of the RSM class
rsm = RSM(diff_data,energy,delta,gamma,num_angle,th_step,pix,det_dist,offset,stack,elem_list)
# transform from detector coordinates to crystal coordinates
rsm.calcRSM('cryst',data_store,desired_workers=10)
# calculate strain
# 'com', center of mass, is a simple algorithm to calculate the strain. Note: There is an abitrary offset
rsm.calcSTRAIN(method) 
# show results
rsm.disp()
# save results
rsm.save(output_path)
# also save the entire object
save_file = f"{output_path}all_data.obj"
pickle.dump(rsm, open(save_file,'wb'),protocol = 4)

In [ ]:
rsm.run_interactive(scale='linear')

In [ ]:
qz = np.sum(rsm.qxz_data,-2)
plt.figure()
 
x = rsm.zq[0,0,:]
y = qz[35,35,:]
popt, fit_y = fit_peaks(x, y,peak_type='lorentzian',n_peaks=2)
plt.plot(x,y,'black',x,fit_y,'red')
plt.show(block=False)

In [ ]:
method = {'fit_type': 'peak',# fitting method: center of mass, 'com' or 'peak'
         'shape': 'gaussian', # peak shape: 'gaussian', 'lorentzian', or 'voigt'
         'n_peaks': [1,1,1]} # number of peaks in each direction, qx, qy, and qz

rsm.calcSTRAIN(method) 
# show results
rsm.disp()

In [ ]:
rsm.disp()